## Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2 

## Import image

In [7]:
# X is the noisy image
noise_image = cv2.imread('images/noise1.png')
noise_image = cv2.cvtColor(noise_image, cv2.COLOR_BGR2RGB)

r, g, b = noise_image[:, :, 0], noise_image[:, :, 1], noise_image[:, :, 2]


### Median filter

In [ ]:
def median_filter(image, kernel_size):
    if kernel_size % 2 == 0:
        raise ValueError("Kernel size must be odd.")
    if kernel_size < 1:
        raise ValueError("Kernel size must be greater than 0.")
    if not isinstance(image, np.ndarray):
        raise TypeError("Image must be a numpy array.")
    if image.ndim != 2:
        raise ValueError("Image must be a 2D array.")
    if not isinstance(kernel_size, int):
        raise TypeError("Kernel size must be an integer.")
    


    pad_width = kernel_size // 2
    # Pad the image to handle borders with the same pixel values
    # as the nearest pixel
    # This is done to avoid introducing artifacts at the borders
    # of the image when applying the filter
    # The mode 'edge' pads with the nearest pixel value
    padded_image = np.pad(image, pad_width, mode='edge')
    filtered_image = np.zeros_like(image)

    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            window = padded_image[i:i + kernel_size, j:j + kernel_size]
            filtered_image[i, j] = np.median(window)

    return filtered_image

### Adaptive process

In [ ]:
def theta_first_indices_k_smallest_elements(v, k):
    if not isinstance(v, np.ndarray):
        raise TypeError("Input must be a numpy array.")
    if not isinstance(k, int):
        raise TypeError("k must be an integer.")
    if k <= 0:
        raise ValueError("k must be greater than 0.")
    if k > len(v):
        raise ValueError("k must be less than or equal to the length of the array.")
    if v.ndim != 1:
        raise ValueError("Input array must be 1D.")
    if len(v) == 0:
        raise ValueError("Input array must not be empty.")

    # Get the indices of the sorted array
    sorted_indices = np.argsort(v)
    # Get the sorted values
    sorted_values = np.sort(v)

    # Return the first k indices
    return sorted_indices[:k]
 
    

def adaptive_processor(X, Y, E1, a=1,b=1):
    if (a<=0):
        raise ValueError("a must be greater than 0.")
    if (b<0):
        raise ValueError("b must be greater than or equal to 0.")
    if (E1.shape != Y.shape):
        raise ValueError("error_index_matrix must have the same shape as filtered_image.")
    if (not isinstance(Y, np.ndarray)):
        raise TypeError("filtered_image must be a numpy array.")
    if (not isinstance(E1, np.ndarray)):
        raise TypeError("error_index_matrix must be a numpy array.")
    if (Y.ndim != 2):
        raise ValueError("filtered_image must be a 2D array.")
    if (E1.ndim != 2):
        raise ValueError("error_index_matrix must be a 2D array.")
    if (Y.shape[0] != E1.shape[0]):
        raise ValueError("filtered_image and error_index_matrix must have the same number of rows.")
    if (Y.shape[1] != E1.shape[1]):
        raise ValueError("filtered_image and error_index_matrix must have the same number of columns.")
    M,N = Y.shape 
    Lmbda_columnwise = np.sum(E1) / N
    lmbda = np.sum(E1, axis=0) / M 
    eta = a*np.std(lmbda) 
    newY =np.copy(Y)

    for n in range(N): 
        if (lmbda[n]-Lmbda_columnwise) > eta:
            e = X[:,n] - Y[:,n]
            K = lmbda[n] - Lmbda_columnwise + b*np.std(lmbda) 
            v=theta_first_indices_k_smallest_elements(np.transpose(e), K) 
            for m in v:
                newY[m,n] = X[m,n] 
            
    return newY


### Omega operator

In [ ]:
def omega_operator(X):
    if not isinstance(X, np.ndarray):
        raise TypeError("Input must be a numpy array.")
    if X.ndim != 2:
        raise ValueError("Input array must be 2D.")
    if len(X) == 0:
        raise ValueError("Input array must not be empty.")

    M, N = X.shape
    omega = np.zeros((M, N))

    for i in range(M):
        for j in range(N):
            if (X[i,j] !=0):
                omega[i, j] = 1
            else:
                omega[i, j] = 0
    return omega

## Alogrithm

In [ ]:
def filter(X):
    W1 = 3
    W2 = 5
    Y = median_filter(noise_image, W1)
    E1 = omega_operator(X-Y)
    
    Y1 = adaptive_processor(X, Y, E1)
    E2 = omega_operator(Y1-Y)

    Z = median_filter(Y1, W2) 
    for m in range(Z.shape[0]):
        for n in range(Z.shape[1]):
            if E2[m,n] == 1:
                Z[m,n] = X[m,n] 
    return Z
    